# Pipeline

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

dataset = pd.read_csv('../Data/SMSSpamCollection.tsv', sep='\t', names=['label', 'message'])

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

dataset['message_length'] = dataset['message'].apply(lambda x: len(x) - x.count(" "))
dataset['punct%'] = dataset['message'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(dataset['message'])

X_features = pd.concat([dataset['message_length'], dataset['punct%'], pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vect.get_feature_names_out())], axis=1)
X_features.head()

,message_length,punct%,,0,008704050406,0089mi,0121,01223585236,01223585334,0125698789,...,zindgi,zoe,zogtoriu,zoom,zouk,zyada,é,ü,üll,〨ud
0,160,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Split training/testing data

In [2]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X_features, dataset['label'], test_size=0.2)

# Grid Search Gradient Boosting Model

In [4]:
from sklearn.ensemble import GradientBoostingClassifier

In [5]:
def train_gb(estimators, maximum_depth, learn_rate):
    gb = GradientBoostingClassifier(n_estimators=estimators, max_depth=maximum_depth, learning_rate=learn_rate)
    gb_model =  gb.fit(X_train, y_train)
    y_pred = gb_model.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth {} / LR: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        estimators, maximum_depth, learn_rate, round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)
    ))

In [6]:
for n_est in [50, 100, 150]:
    for max_depth in [3, 7, 11, 15]:
        for lr in [0.01, 0.1, 1]:
            train_gb(n_est, max_depth, lr)

C:\Users\rueip\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Est: 50 / Depth 3 / LR: 0.01 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.88
Est: 50 / Depth 3 / LR: 0.1 ---- Precision: 0.951 / Recall: 0.724 / Accuracy: 0.962
Est: 50 / Depth 3 / LR: 1 ---- Precision: 0.866 / Recall: 0.769 / Accuracy: 0.958
Est: 50 / Depth 7 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.007 / Accuracy: 0.881
Est: 50 / Depth 7 / LR: 0.1 ---- Precision: 0.902 / Recall: 0.821 / Accuracy: 0.968
Est: 50 / Depth 7 / LR: 1 ---- Precision: 0.865 / Recall: 0.813 / Accuracy: 0.962


C:\Users\rueip\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Est: 50 / Depth 11 / LR: 0.01 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.88
Est: 50 / Depth 11 / LR: 0.1 ---- Precision: 0.899 / Recall: 0.799 / Accuracy: 0.965
Est: 50 / Depth 11 / LR: 1 ---- Precision: 0.88 / Recall: 0.821 / Accuracy: 0.965


C:\Users\rueip\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Est: 50 / Depth 15 / LR: 0.01 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.88
Est: 50 / Depth 15 / LR: 0.1 ---- Precision: 0.879 / Recall: 0.813 / Accuracy: 0.964
Est: 50 / Depth 15 / LR: 1 ---- Precision: 0.876 / Recall: 0.843 / Accuracy: 0.967
Est: 100 / Depth 3 / LR: 0.01 ---- Precision: 0.958 / Recall: 0.515 / Accuracy: 0.939
Est: 100 / Depth 3 / LR: 0.1 ---- Precision: 0.964 / Recall: 0.799 / Accuracy: 0.972
Est: 100 / Depth 3 / LR: 1 ---- Precision: 0.955 / Recall: 0.791 / Accuracy: 0.97
Est: 100 / Depth 7 / LR: 0.01 ---- Precision: 0.947 / Recall: 0.672 / Accuracy: 0.956
Est: 100 / Depth 7 / LR: 0.1 ---- Precision: 0.917 / Recall: 0.828 / Accuracy: 0.97
Est: 100 / Depth 7 / LR: 1 ---- Precision: 0.865 / Recall: 0.813 / Accuracy: 0.962
Est: 100 / Depth 11 / LR: 0.01 ---- Precision: 0.893 / Recall: 0.746 / Accuracy: 0.959
Est: 100 / Depth 11 / LR: 0.1 ---- Precision: 0.919 / Recall: 0.843 / Accuracy: 0.972
Est: 100 / Depth 11 / LR: 1 ---- Precision: 0.949 / Recall: 0.836 / Accu